## Notebook for Production of DE1 tables
- 58 tables from demographics data
- Population numbers by 5-year age intervals

### Import packages & functions

In [ ]:
library(haven)
library(PxWebApiData)
library(tidyverse)
library(klassR)

In [ ]:
# define paths

functions_path <- paste(getwd(), "functions/", sep = "/")

# source function and correspondence

source(paste(functions_path, "functions.r", sep = "/"))
source(paste(functions_path, "correspondence.r", sep = "/"))

### Statbank data import

In [ ]:
# import data for population variables

df_DE1 <- ApiData(07459,
                  Region = c("0301", "1103", "4601", "5001", "3005"),
                  Alder = list("agg:FemAarigGruppering",
                               c("F00-04", "F05-09", "F10-14",
                                 "F15-19", "F20-24", "F25-29",
                                 "F30-34", "F35-39", "F40-44",
                                 "F45-49", "F50-54", "F55-59",
                                 "F60-64", "F65-69", "F70-74",
                                 "F75-79", "F80-84", "F85-89",
                                 "F90-94", "F95-99", "F100G5+")),
                  Tid = "2022")[[2]]

In [ ]:
# import data for median

df_DE1073V <- ApiData(13536,
                      Region = list("vs:Kommune", c("0301",
                                                    "1103",
                                                    "4601",
                                                    "5001",
                                                    "3005")),
                      ContentsCode = "Medianalder",
                      Tid = "2022")[[2]]

### Data processing
1. Add a column with municipality names onto the dataframe based on region code (klass = 131)
2. Load a correspondence table between city name and city code
3. Merge the dataframe and city correspondence 

In [ ]:
# import klass correspondence for city names

df_DE1$name <- ApplyKlass(df_DE1$Region, klass = 131)
df_DE1073V$name <- ApplyKlass(df_DE1073V$Region, klass = 131)

city_correspondence <- GetKlass(550) %>%
                           select(code, name)

In [ ]:
# merge klass correspondence with dataframe

df_DE1 <- merge.data.frame(df_DE1, city_correspondence,
                           by = "name")

df_DE1073V <- merge.data.frame(df_DE1073V, city_correspondence,
                               by = "name")

### Summary tables

#### Part 1:
This section of code produces tables that require summing over several rows of the imported age and gender separated data. In particular, 
  
DE1001V - DE1003V: Population on the 1st of January (total, male and female);  
DE1073V: Median population age;  
DE1137V - DE1139V: Population on the 1st of January, 85 years or over.

In [ ]:
# DE1001V - DE1003V

DE1001V_2022 <- df_DE1 %>%
                   group_by(code) %>%
                   summarise(Value = sum(value))

DE1002V_2022 <- sum_rows_DE1(Kjonn, "1")

DE1003V_2022 <- sum_rows_DE1(Kjonn, "2")

In [ ]:
# DE1073V

DE10073_2022 <- df_DE1073V %>%
                    select(code, value) %>%
                    rename(Value = value)

In [ ]:
# DE1137V - DE1139V

DE1137V_2022 <- df_DE1 %>%
                    filter(Alder %in% c("F85-89",
                                        "F90-94",
                                        "F95-99",
                                        "F100G5+")) %>%
                    group_by(code) %>%
                    summarise(Value = sum(value))

DE1138V_2022 <- df_DE1 %>%
                    filter(Alder %in% c("F85-89",
                                        "F90-94",
                                        "F95-99",
                                        "F100G5+"),
                           Kjonn == "1") %>%
                    group_by(code) %>%
                    summarise(Value = sum(value))

DE1139V_2022 <- df_DE1 %>%
                    filter(Alder %in% c("F85-89",
                                        "F90-94",
                                        "F95-99",
                                        "F100G5+"),
                          Kjonn == '2') %>%
                    group_by(code) %>%
                    summarise(Value = sum(value))

#### Part 2.1 Totals:
This section of code loops over the variables defined in correspondence.r and creates all 'total' tables. 

In [ ]:
# loop over variable_names with totals and create summary tables using sum_rows

for (i in seq(1, nrow(variable_correspondence), 3)) {
    assign(
        paste(as.character(variable_correspondence$var_name[i]),
              "2022",
              sep = "_"),
        sum_rows_DE1(Alder, as.character(variable_correspondence$var_age[i])))
}

#### Part 2.2. Gender separated tables
Note, for the gender seperated tables we do not have to sum over rows, but simply filter at the age group and gender. This is done using the function: filter_rows, and specifying *age_group* as the first variable and *gender_code* as the second.

In [ ]:
# set index to be all values that are not totals

index <- setdiff(1:nrow(variable_correspondence),
                 seq(1, nrow(variable_correspondence), 3))

In [ ]:
# loop over index and create summary tables using filter_rows

for (i in index) {
    assign(
        paste(as.character(variable_correspondence$var_name[i]),
              "2022",
              sep = "_"),
        filter_rows_DE1(as.character(variable_correspondence$var_age[i]),
                    as.character(variable_correspondence$var_gender[i])))
}

### Table formatting

In [ ]:
my_list <- ls(pattern = "^[A-Z]{2}.*_2022")

In [ ]:
for (table in my_list) {
    assign(paste(table), table_formatting(table))
}

In [ ]:
ls(pattern = "^[A-Z]{2}.*_2022")

## Export
Add code for exporting to csv (this is the current solution for the presentation).

In [ ]:
# create an empty list to fill in subsequent loop

DE_tables <- list()

# looping over variables/dfs specified by pattern, before appending to list

for (i in ls(pattern = "^DE.*2022")) {
    x <- get(i)
    DE_tables[[i]] <- x
    }

In [ ]:
# rbind the tables

DE_2022 <- do.call("rbind", DE_tables)

In [ ]:
# export to Linux

# write.csv(DE_2022, "/ssb/stamme01/bediv/project/CITYSTATISTICS/DE_2022.csv", row.names = FALSE)